In [1]:
# Global inversion with empymod

%matplotlib notebook

In [2]:
import empymod
import numpy as np
#import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed

In [3]:

# Receivers geometry

offsets = np.array([2, 4, 8]) # in meters
dip = np.array([0, 90])

Hreceivers = [offsets, offsets*0, 0, 0, 0]
Vreceivers = [offsets, offsets*0, 0, 0, 90]

# Source geometry

Hsource = [0, 0, 0 ,0 , 0]
Vsource = [0, 0, 0, 0, 90]

# Frequency

freq = 9000

In [4]:
# True data

res = [2e14, 10, 20, 10]
depth = [0, 2, 5]

HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop')
VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop')
PRP = empymod.loop(Hsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop')

Zdata = np.hstack((HCP, VCP, PRP))



:: empymod END; runtime = 0:00:01.227318 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.007817 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.007009 :: 1 kernel call(s)



In [ ]:
print(HCP)
print(VCP)
print(PRP)

In [5]:
# sampling of depth and conductivities
nsl = 1

s0 = -2 # minimum conductivity in S/m
s1 = -0.8 # maximum conductivity in S/m
# conductivities array
conds = np.logspace(s0, s1, nsl)

th0 = 0.1 # minimum thickness in m
th1 = 5   # maximum thickness in m
# thickness array
thicks = np.linspace(th0, th1, nsl)

# Array to store values

#Zcube = np.zeros((nsl, nsl, nsl, nsl, nsl, 9), dtype = 'complex') # 9 coil geometries, 5 parameters 

In [ ]:
# Loop to create hypercube

startTime = time.time()

err = 1

for is1 in range(0, nsl):
    res[1] = 1/conds[is1] # set resistivity of first layer
    
    for is2 in range(0, nsl):
        res[2] = 1/conds[is2] # set resistivity of second layer
        
        for is3 in range(0, nsl): 
            res[3] = 1/conds[is3] # set resistivity of third layer
            
            for it1 in range(0, nsl):
                depth[1] = thicks[it1] # set thickness of first layer
                
                for it2 in range(0, nsl):
                    depth[2] = depth[1] + thicks[it2] # set thickness of second layer
                    
                    # Compute fields
                    
                    HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
                    VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
                    PRP = empymod.loop(Hsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
                    
                    # Store in hypercube
                    
                    Zcube[is1, is2, is3, it1, it2, 0:3] = HCP
                    Zcube[is1, is2, is3, it1, it2, 3:6] = VCP
                    Zcube[is1, is2, is3, it1, it2, 6:9] = PRP
                    
                    Z = np.hstack((HCP, VCP, PRP))
                    
                    # Calculate amplitude of difference
                    
                    nZdiff = np.abs(Z - Zdata) **2 / np.abs(Zdata)**2

                    merr = np.log10(np.sqrt(np.sum(nZdiff)))
                    
                    if merr < err: # until error increases
                        # set model values
                        ms1 = is1
                        ms2 = is2
                        ms3 = is3
                        mt1 = it1
                        mt2 = it2
                        err = merr
                        

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

print('Best model is found at an error of ' + str(10**err) +' for')
print('sigma_1 = '+ str(conds[ms1])+ ' S/m, d_1 ' + str(thicks[mt1]) + 'm')
print('sigma_2 = '+ str(conds[ms2])+ ' S/m, d_2 ' + str(thicks[mt1] + thicks[mt2]) + 'm')
print('sigma_3 = '+ str(conds[ms3])+ ' S/m')

In [6]:
def forward_parallel(is1, is2, is3, it1, it2):
    time.sleep(0.1)
    
    res[1] = 1/conds[is1] # set resistivity of first layer
    res[2] = 1/conds[is2] # set resistivity of second layer
    res[3] = 1/conds[is3] # set resistivity of third layer
    depth[1] = thicks[it1] # set thickness of first layer
    depth[2] = depth[1] + thicks[it2] # set thickness of second layer

    # Compute fields

    HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
    VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
    PRP = empymod.loop(Hsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)

    # Store in hypercube

   # Zcube[is1, is2, is3, it1, it2, 0:3] = HCP
   # Zcube[is1, is2, is3, it1, it2, 3:6] = VCP
   # Zcube[is1, is2, is3, it1, it2, 6:9] = PRP

    Z = np.hstack((HCP, VCP, PRP))

    # Calculate amplitude of difference

    #nZdiff = np.abs(Z - Zdata) **2 / np.abs(Zdata)**2

    #merr = np.log10(np.sqrt(np.sum(nZdiff)))

    return is1, Z

In [12]:
h = 51

In [13]:
starttime = time.time()
r = Parallel(n_jobs=4, verbose=10)(delayed(forward_parallel)(i, j, k, m, n) 
                                    for i in range(h) for j in range(h) for k in range (h) for m in range(h) for n in range(h))

endtime = time.time() - starttime
print('Execution time parallel is:', endtime)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    3.4s


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
r